# More processing data with pandas 

Along the way we will cover a number of useful techniques in pandas including:

- renaming columns
- iterating data frame rows and applying functions
- data aggregation
- repeating the analysis task for several input files

## Input data

In the lesson this week we are using simulated data for Sweden. You will be working with data from a total of 5 different weather observation stations in Sweden.

## Reading the data

In order to get started, let's first import pandas: 

In [1]:
import pandas as pd

At this point, we can already have a quick look at the data file `Stockholm.txt` for Stockholm and how it is structured. We can notice at least two things we need to consider when reading in the data:

```{admonition} Input data structure
- **Delimiter:** The data are **separated with a varying amount of spaces**. If you check out the documentation for the [read_csv() method](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html), you can see that there are two different ways of doing this. We can use either `sep='\s+'` or `delim_whitespace=True` (but not both at the same time). In this case, we prefer to use `delim_whitespace` parameter.

- **No Data values:** No data values in the Simulated Data data are coded with varying number of `*`. We can tell pandas to consider those characters as NaNs by specifying `na_values=['*', '**', '***', '****', '*****', '******']`.
```

In [2]:
# Define absolute path to the file
fp = r"D:/Teaching_Development/Python_1_course_VT24/missing lecture/dataset/Stockholm_continuous_final.txt"  

In [3]:
# Read data using varying amount of spaces as separator and specifying * characters as NoData values
# And don't worry about the warning message generated when running this cell
data = pd.read_csv(
    fp, delim_whitespace=True, na_values=["*", "**", "***", "****", "*****", "******"]
)

In [4]:
data.head()

,USAF,WBAN,Temperature,Precipitation,WindSpeed,Humidity,City,YRMODAHRMN
0,221958,28030,82.530449,35.871553,3.502463,59.251273,Stockholm,201801010926
1,771155,92838,60.016636,80.443721,13.455292,75.763007,Stockholm,201801020720
2,231932,94717,76.490117,27.875987,4.410085,99.744067,Stockholm,201801030828
3,465838,65658,46.741270,21.070496,4.364151,66.115737,Stockholm,201801041432
4,359178,48638,51.122914,95.744802,17.481435,95.241484,Stockholm,201801052236


All seems ok. However, we won't be needing all of the 8 columns for detecting warm temperatures in April. We can check all column names by running `data.columns`:

In [5]:
data.columns

Index(['USAF', 'WBAN', 'Temperature', 'Precipitation', 'WindSpeed', 'Humidity',
       'City', 'YRMODAHRMN'],
      dtype='object')

A description for all these columns is available in the metadata file [data/3505doc.txt](SwedishDataDescription.txt). 

### Reading in the data once again

This time, we will read in only some of the columns using the `usecols` parameter. Let's read in columns that might be somehow useful to our analysis, or at least that contain some values that are meaningful to us, including the station name, timestamp, and data about wind and temperature: `'Date','Temperature', 'Precipitation', 'City'`

In [6]:
# Read in only selected columns
data = pd.read_csv(
    fp,
    delim_whitespace=True,
    usecols=["YRMODAHRMN","Temperature", "Precipitation","City"],
    na_values=["*", "**", "***", "****", "*****", "******"],
)

# Check the dataframe
data.head()

,Temperature,Precipitation,City,YRMODAHRMN
0,82.530449,35.871553,Stockholm,201801010926
1,60.016636,80.443721,Stockholm,201801020720
2,76.490117,27.875987,Stockholm,201801030828
3,46.741270,21.070496,Stockholm,201801041432
4,51.122914,95.744802,Stockholm,201801052236


Okay so we can see that the data was successfully read to the DataFrame and we also seemed to be able to convert the asterisk (\*) characters into `NaN` values. 

## Renaming columns

Let's rename the columns  using the [rename](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rename.html) function. In order to change the column names, we need to tell pandas how we want to rename the columns using a dictionary that lists old and new column names

Let's first check again the current column names in our DataFrame:

In [7]:
data.columns

Index(['Temperature', 'Precipitation', 'City', 'YRMODAHRMN'], dtype='object')

We can define the new column names using a [dictionary](https://docs.python.org/3/tutorial/datastructures.html#dictionaries) where we list `key: value` pairs, in which the original column name (the one which will be replaced) is the key and the new column name is the value.

```{admonition} Dictionaries
A [dictionary](https://docs.python.org/3/tutorial/datastructures.html#dictionaries) is a specific type of data structure in Python for storing key-value pairs. In this course, we will use dictionaries mainly when renaming columns in a pandas DataFrame, but dictionaries are useful for many different purposes! For more information about Python dictionaries, check out [this tutorial](https://realpython.com/python-dicts/).
```

Let's change the following:

- `Temperature` to `Temp_Average`
- `Precipitation` to `PRECIP`
- `City` to `Location`
- `YRMODAHRMN` to `Date`



In [8]:
# Create the dictionary with old and new names
new_names = {"Temperature": "Temp_Average", "Precipitation": "PRECIP", "City": "Location","YRMODAHRMN": "Date"}

# Let's see what the variable new_names look like
new_names

{'Temperature': 'Temp_Average',
 'Precipitation': 'PRECIP',
 'City': 'Location',
 'YRMODAHRMN': 'Date'}

In [9]:
# Check the data type of the new_names variable
type(new_names)

dict

From above we can see that we have successfully created a new dictionary. 

Now we can change the column names by passing that dictionary using the parameter `columns` in the `rename()` function:

In [10]:
# Rename the columns
data = data.rename(columns=new_names)

# Print the new columns
print(data.columns)

Index(['Temp_Average', 'PRECIP', 'Location', 'Date'], dtype='object')


Perfect, now our column names are easier to understand and use. 

### Check your understanding

The temperature values in our data files are again in Fahrenheit. As you might guess, we will soon convert these temperatures in to Celsius. In order to avoid confusion with the columns, let's rename the column `TEMP` to `TEMP_F`. Let's also rename `USAF` to `STATION_NUMBER`.

In [11]:
# Solution
# Create the dictionary with old and new names
new_names = { "Temp_Average": "Temp_Average_F"}

# Rename the columns
data = data.rename(columns=new_names)

# Check the output
data.head()

,Temp_Average_F,PRECIP,Location,Date
0,82.530449,35.871553,Stockholm,201801010926
1,60.016636,80.443721,Stockholm,201801020720
2,76.490117,27.875987,Stockholm,201801030828
3,46.741270,21.070496,Stockholm,201801041432
4,51.122914,95.744802,Stockholm,201801052236


## Data properties

As we learned last week, it's always a good idea to check basic properties of the input data before proceeding with the data analysis. Let's check the:

- Number of rows and columns

In [12]:
data.shape

(1096, 4)

- Top and bottom rows

In [13]:
data.head()

,Temp_Average_F,PRECIP,Location,Date
0,82.530449,35.871553,Stockholm,201801010926
1,60.016636,80.443721,Stockholm,201801020720
2,76.490117,27.875987,Stockholm,201801030828
3,46.741270,21.070496,Stockholm,201801041432
4,51.122914,95.744802,Stockholm,201801052236


In [14]:
data.tail()

,Temp_Average_F,PRECIP,Location,Date
1091,63.515987,36.739073,Stockholm,202012271736
1092,66.034901,8.701722,Stockholm,202012281946
1093,63.862335,55.678491,Stockholm,202012292059
1094,23.679329,84.512085,Stockholm,202012301905
1095,35.556482,79.592639,Stockholm,202012310348


- Data types of the columns

In [15]:
data.dtypes

Temp_Average_F    float64
PRECIP            float64
Location           object
Date                int64
dtype: object

- Descriptive statistics

In [16]:
data.describe()

,Temp_Average_F,PRECIP,Date
count,1058.000000,1054.000000,1.096000e+03
mean,50.213660,49.441831,2.019068e+11
std,20.693548,28.811276,8.177353e+07
min,14.112688,0.001163,2.018010e+11
25%,32.680722,24.776389,2.018100e+11
50%,50.781486,49.647709,2.019070e+11
75%,68.095460,74.216397,2.020040e+11
max,85.957788,99.889261,2.020123e+11


Here we can see that there are varying number of observations per column (look at the `count` row above), because some of the columns have missing values.

## Using your own functions in pandas 

Now it's again time to convert temperatures from Fahrenheit to Celsius! Yes, we have already done this many times before, but this time we will learn how to apply our own functions to data in a pandas DataFrame.

First, we will define a function for the temperature conversion, and then we will apply this function for each Fahrenheit value on each row of the DataFrame. The output celsius values will be stored in a new column called `TEMP_C`.

To begin we will see how we can apply the function row-by-row using a `for` loop and then we will learn how to apply the function to all rows more efficiently all at once.

### Defining the function

For both of these approaches, we first need to define our function to convert temperature from Fahrenheit to Celsius.

In [17]:
def fahr_to_celsius(temp_fahrenheit):
    """Function to convert Fahrenheit temperature into Celsius.

    Parameters
    ----------

    temp_fahrenheit: int | float
        Input temperature in Fahrenheit (should be a number)

    Returns
    -------

    Temperature in Celsius (float)
    """

    # Convert the Fahrenheit into Celsius
    converted_temp = (temp_fahrenheit - 32) / 1.8

    return converted_temp

To make sure everything is working properly, let's test the function with a known value.

In [18]:
fahr_to_celsius(32)

0.0

Let's also print out the first rows of our data frame to see our input data before further processing.

In [19]:
data.head()

,Temp_Average_F,PRECIP,Location,Date
0,82.530449,35.871553,Stockholm,201801010926
1,60.016636,80.443721,Stockholm,201801020720
2,76.490117,27.875987,Stockholm,201801030828
3,46.741270,21.070496,Stockholm,201801041432
4,51.122914,95.744802,Stockholm,201801052236


### Iterating over rows

We can use the function one row at a time using a `for` loop and the [iterrows()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.iterrows.html) method. This will allow us to iterate row by row using `iterrows()` in a `for` loop to repeat a given process *for each row in a pandas DataFrame*. Please note that iterating over rows is a rather inefficient approach, but it is still useful to understand the logic behind the iteration.

When using the `iterrows()` method it is important to understand that `iterrows()` accesses not only the values of one row, but also the `index` of the row as well. 

Let's start with a simple for loop that goes through each row in our DataFrame.

```{note}
We use single quotes to select the column `Temp_Average_F` of the row in the example below. This is because using double quotes would result in a `SyntaxError` since Python would interpret this as the end of the string for the `print()` function.
```

In [20]:
# Iterate over the rows
for idx, row in data.iterrows():

    # Print the index value
    print(f"Index: {idx}")

    # Print the row
    print(f"Temp F: {row['Temp_Average_F']}\n")

    break

Index: 0
Temp F: 82.53044855047321



```{admonition} Breaking a loop
When developing code in a `for` loop, you do not always need to go through the entire loop in order to test things out. 
The [break](https://docs.python.org/3/reference/simple_stmts.html#break) statement in Python terminates the current loop whereever it is placed and we can use it here just to check out the values on the first row (based on the first iteration in the `for` loop.
This can be helpful when working with a large data file or dataset, because you might not want to print thousands of values to the screen!
For more information, check out [this tutorial](https://www.tutorialspoint.com/python/python_break_statement.htm).
```

We can see that the `idx` variable indeed contains the index value at position 0 (the first row) and the `row` variable contains all the data from that given row stored as a pandas `Series`.

Let's now create an empty column `TEMP_C` for the Celsius temperatures and update the values in that column using the `fahr_to_celsius` function we defined earlier.

In [21]:
# Create an empty float column for the output values
data["TEMP_C"] = 0.0

# Iterate over the rows
for idx, row in data.iterrows():

    # Convert the Fahrenheit to Celsius
    celsius = fahr_to_celsius(row["Temp_Average_F"])

    # Update the value of 'Celsius' column with the converted value
    data.at[idx, "TEMP_C"] = celsius

```{admonition} Reminder: .at or .loc?
Here, you could also use `data.loc[idx, new_column] = celsius` to achieve the same result. 
    
If you only need to access a single value in a DataFrame, [DataFrame.at](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.at.html) is faster than [DataFrame.loc](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.loc.html), which is designed for accessing groups of rows and columns. 
```

Finally, let's see how our DataFrame looks like now after the calculations above.

In [22]:
data.head()

,Temp_Average_F,PRECIP,Location,Date,TEMP_C
0,82.530449,35.871553,Stockholm,201801010926,28.072471
1,60.016636,80.443721,Stockholm,201801020720,15.564798
2,76.490117,27.875987,Stockholm,201801030828,24.716732
3,46.741270,21.070496,Stockholm,201801041432,8.189594
4,51.122914,95.744802,Stockholm,201801052236,10.623841


### Applying the function

pandas DataFrames and Series have a dedicated method `.apply()` for applying functions on columns (or rows!). When using `.apply()`, we pass the function name (without parentheses!) as an argument to the `apply()` method. Let's start by applying the function to the `TEMP_F` column that contains the temperature values in Fahrenheit.

In [23]:
data["Temp_Average_F"].apply(fahr_to_celsius)

0       28.072471
1       15.564798
2       24.716732
3        8.189594
4       10.623841
          ...    
1091    17.508882
1092    18.908278
1093    17.701297
1094    -4.622595
1095     1.975824
Name: Temp_Average_F, Length: 1096, dtype: float64

The results look logical, so we can store them permanently into the `TEMP_C` column (overwriting the old values).

In [24]:
data["TEMP_C"] = data["Temp_Average_F"].apply(fahr_to_celsius)

We can also apply the function on several columns at once. Furthermore, we can re-order the dataframe at the same time.

### Check your understanding

Convert `'Temp_Average_F'`, to Celsius by applying the function like we did above and store the outputs to  new columns `'TEMP_C'`.

In [25]:
# Solution
data[["TEMP_C"]] = data[["Temp_Average_F"]].apply(
    fahr_to_celsius
)

```{note}
Applying the function on all columns using `data.apply(fahr_to_celsius)` would not give an error for the data we are using, but the results also do not make much sense for columns where input data are something other than Fahrenheit temperatures.
```

You might also notice that our conversion function would also allow us to 
pass one column or the entire dataframe as a parameter. For example, like this: `fahr_to_celsius(data["TEMP_F"])`. However, the code is perhaps easier to follow when using the apply method.

Let's check now take a look at the DataFrame contents.

In [26]:
data.head(10)

,Temp_Average_F,PRECIP,Location,Date,TEMP_C
0,82.530449,35.871553,Stockholm,201801010926,28.072471
1,60.016636,80.443721,Stockholm,201801020720,15.564798
2,76.490117,27.875987,Stockholm,201801030828,24.716732
3,46.741270,21.070496,Stockholm,201801041432,8.189594
4,51.122914,95.744802,Stockholm,201801052236,10.623841
5,49.196954,0.886329,Stockholm,201801061435,9.553863
6,62.014227,99.782086,Stockholm,201801071827,16.674570
7,24.054890,67.682130,Stockholm,201801080922,-4.413950
8,16.158098,82.846930,Stockholm,201801090104,-8.801056
9,36.170956,29.461940,Stockholm,201801101754,2.317198


```{admonition} Should I use .iterrows() or .apply()?
We are teaching the `.iterrows()` method because it helps to understand the structure of a DataFrame and the process of looping through DataFrame rows. However, using `.apply()` is often more efficient in terms of execution time. 

At this point, the most important thing is that you understand what happens when you are modifying the values in a pandas DataFrame. When doing the course exercises, either of these approaches is ok!
```

## Parsing dates

As part of this lesson, we eventually want to group our data based on month in order to see whether the temperatures in April of 2019 were warmer than average. Currently, the date and time information is stored in the column `Date` (which was originally titled `YR--MODAHRMN`:

`YR--MODAHRMN = YEAR-MONTH-DAY-HOUR-MINUTE IN GREENWICH MEAN TIME (GMT)`

Let's have a closer look at the date and time information we have by checking the values in that column, and their data type.

In [27]:
data["Date"].head(10)

0    201801010926
1    201801020720
2    201801030828
3    201801041432
4    201801052236
5    201801061435
6    201801071827
7    201801080922
8    201801090104
9    201801101754
Name: Date, dtype: int64

In [28]:
data["Date"].tail(10)

1086    202012220051
1087    202012230604
1088    202012240310
1089    202012251837
1090    202012261926
1091    202012271736
1092    202012281946
1093    202012292059
1094    202012301905
1095    202012310348
Name: Date, dtype: int64

The `Date` column contains several observations per day (and even several observations per hour). The timestamp for the first observation is `201801010926`, i.e. from 1st of January 2018 , and the timestamp for the latest observation is the last day of December 2020.

In [29]:
data["Date"].dtypes

dtype('int64')

The information in this column is stored as integer values.

We now want to aggregate the data on a monthly level, and in order to do so we need to "label" each row of data based on the month when the record was observed. In order to do this, we need to somehow separate information about the year and month for each row.

We can create these "labels" by making a new column containing information about the month (including the year, but excluding day, hours, and minutes).

Before further taking that step, we should first convert the contents in the `Date` column to a new column with character string values for convenience.

In [30]:
# Convert to string
data["Date_STR"] = data["Date"].astype(str)

### String slicing

Now that we have converted the date and time information into character strings, we next need to "cut" the needed information from the [string objects](https://docs.python.org/3/tutorial/introduction.html#strings). If we look at the latest time stamp in the data (`201910012350`), you can see that there is a systematic pattern `YEAR-MONTH-DAY-HOUR-MINUTE`. Four first characters represent the year, and six first characters are year and month!

In [31]:
date = "201801010926" 
date[0:6]

'201801'

Based on this information, we can slice the correct range of characters from the `TIME_STR` column using the [pandas.Series.str.slice()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.slice.html) method.

In [32]:
# Slice the string
data["YEAR_MONTH"] = data["Date_STR"].str.slice(start=0, stop=6)

# Let's see what we have
data.head()

,Temp_Average_F,PRECIP,Location,Date,TEMP_C,Date_STR,YEAR_MONTH
0,82.530449,35.871553,Stockholm,201801010926,28.072471,201801010926,201801
1,60.016636,80.443721,Stockholm,201801020720,15.564798,201801020720,201801
2,76.490117,27.875987,Stockholm,201801030828,24.716732,201801030828,201801
3,46.741270,21.070496,Stockholm,201801041432,8.189594,201801041432,201801
4,51.122914,95.744802,Stockholm,201801052236,10.623841,201801052236,201801


Nice! Now we have "labeled" the rows based on information about date and time, but only including the year and month in the labels.

### Check your understanding

Create a new column called `'MONTH'` with information about the month of each observation, but excluding the year.

In [33]:
# Solution
# Extract information about month from the TIME_STR column into a new column 'MONTH':
data["MONTH"] = data["Date_STR"].str.slice(start=4, stop=6)

# Check the result
data[["YEAR_MONTH", "MONTH"]]

,YEAR_MONTH,MONTH
0,201801,01
1,201801,01
2,201801,01
3,201801,01
4,201801,01
...,...,...
1091,202012,12
1092,202012,12
1093,202012,12
1094,202012,12


## Aggregating data in pandas by grouping

Here, we will learn how to use [pandas.DataFrame.groupby](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html), which is a handy method for combining large amounts of data and computing statistics for subgroups.

In our case, we will use the groupby method to calculate the average temperatures for each month through these three steps:

  1. Grouping the data based on the year and month
  2. Calculating the average for each month (each group) 
  3. Storing those values into a new DataFrame called `monthly_data`

Before we start grouping the data, let's once again see what our data looks like.

In [34]:
print(f"number of rows: {len(data)}")

number of rows: 1096


In [35]:
data.head()

,Temp_Average_F,PRECIP,Location,Date,TEMP_C,Date_STR,YEAR_MONTH,MONTH
0,82.530449,35.871553,Stockholm,201801010926,28.072471,201801010926,201801,01
1,60.016636,80.443721,Stockholm,201801020720,15.564798,201801020720,201801,01
2,76.490117,27.875987,Stockholm,201801030828,24.716732,201801030828,201801,01
3,46.741270,21.070496,Stockholm,201801041432,8.189594,201801041432,201801,01
4,51.122914,95.744802,Stockholm,201801052236,10.623841,201801052236,201801,01


We have quite a few rows of weather data. Our goal is to create an aggreated data frame that would have only one row per month.

To condense our data to monthly average values we can group our data based on the unique year and month combinations.

In [36]:
grouped = data.groupby("YEAR_MONTH")

````{note}
It is also possible to create combinations of years and months on-the-fly when grouping the data:
    
```python
# Group the data 
grouped = data.groupby(['YEAR', 'MONTH'])
```
````

Now, let's explore the new variable `grouped`.

In [37]:
type(grouped)

pandas.core.groupby.generic.DataFrameGroupBy

In [38]:
len(grouped)

36

We have a new object with type `DataFrameGroupBy` with 36 groups. In order to understand what just happened, let's also check the number of unique year and month combinations in our data.

In [39]:
data["YEAR_MONTH"].nunique()

36

Length of the grouped object should be the same as the number of unique values in the column we used for grouping. For each unique value, there is a group of data.

Let's explore our grouped data even further. 

We can check the "names" of each group.

In [40]:
# Next line will print out all 601 group "keys"
# This is commented out here to avoid excess output to the course website
grouped.groups.keys()

dict_keys(['201801', '201802', '201803', '201804', '201805', '201806', '201807', '201808', '201809', '201810', '201811', '201812', '201901', '201902', '201903', '201904', '201905', '201906', '201907', '201908', '201909', '201910', '201911', '201912', '202001', '202002', '202003', '202004', '202005', '202006', '202007', '202008', '202009', '202010', '202011', '202012'])

### Accessing data for one group

Let us now check the contents for the group representing January 2018 (the name of that group is `201801`. We can get the values of that hour from the grouped object using the `get_group()` method.

In [41]:
# Specify a month (as character string)
month = "201801"

# Select the group
group1 = grouped.get_group(month)

In [42]:
# Let's see what we have
group1

,Temp_Average_F,PRECIP,Location,Date,TEMP_C,Date_STR,YEAR_MONTH,MONTH
0,82.530449,35.871553,Stockholm,201801010926,28.072471,201801010926,201801,01
1,60.016636,80.443721,Stockholm,201801020720,15.564798,201801020720,201801,01
2,76.490117,27.875987,Stockholm,201801030828,24.716732,201801030828,201801,01
3,46.741270,21.070496,Stockholm,201801041432,8.189594,201801041432,201801,01
4,51.122914,95.744802,Stockholm,201801052236,10.623841,201801052236,201801,01
5,49.196954,0.886329,Stockholm,201801061435,9.553863,201801061435,201801,01
6,62.014227,99.782086,Stockholm,201801071827,16.674570,201801071827,201801,01
7,24.054890,67.682130,Stockholm,201801080922,-4.413950,201801080922,201801,01
8,16.158098,82.846930,Stockholm,201801090104,-8.801056,201801090104,201801,01
9,36.170956,29.461940,Stockholm,201801101754,2.317198,201801101754,201801,01


Ahaa! As we can see, a single group contains a DataFrame with values only for that specific month and year. Let's check the DataType of this group.

In [43]:
type(group1)

pandas.core.frame.DataFrame

So, as noted above, one group is a pandas DataFrame! This is really useful, because we can now use all the familiar DataFrame methods for calculating statistics, etc. for this specific group. We can, for example, calculate the average values for all variables using the statistical functions that we have seen already (e.g. mean, std, min, max, median, etc.).

We can do that by using the `mean()` function that we already did during Lesson 5. Let's calculate the mean for following attributes all at once:

- `PRECIP`
- `Temp_Average_F`
- `TEMP_C`
- `MONTH`


In [44]:
# Specify the columns that will be part of the calculation
mean_cols = ["PRECIP", "Temp_Average_F", "TEMP_C"]

# Calculate the mean values all at one go
mean_values = group1[mean_cols].mean()

# Let's see what we have
print(mean_values)

PRECIP            51.752037
Temp_Average_F    55.593683
TEMP_C            13.107602
dtype: float64


Above, we saw how you can access data from a single group. In order to get information about all groups (all months) we can use a `for` loop or methods available in the grouped object.

### For loops and grouped objects

When iterating over the groups in our `DataFrameGroupBy` object it is important to understand that a single group in our `DataFrameGroupBy` actually contains not only the actual values, but also information about the `key` that was used to do the grouping. Hence, when iterating over the data we need to assign the `key` and the values into separate variables.

So, let's see how we can iterate over the groups and print the key and the data from a single group (again using `break` to only see what is happening for the first group).

In [45]:
# Iterate over groups
for key, group in grouped:
    # Print key and group
    print(f"Key:\n {key}")
    print(f"\nFirst rows of data in this group:\n {group.head()}")

    # Stop iteration with break command
    break

Key:
 201801

First rows of data in this group:
    Temp_Average_F     PRECIP   Location          Date     TEMP_C  \
0       82.530449  35.871553  Stockholm  201801010926  28.072471   
1       60.016636  80.443721  Stockholm  201801020720  15.564798   
2       76.490117  27.875987  Stockholm  201801030828  24.716732   
3       46.741270  21.070496  Stockholm  201801041432   8.189594   
4       51.122914  95.744802  Stockholm  201801052236  10.623841   

       Date_STR YEAR_MONTH MONTH  
0  201801010926     201801    01  
1  201801020720     201801    01  
2  201801030828     201801    01  
3  201801041432     201801    01  
4  201801052236     201801    01  


OK, so from here we can see that the `key` contains the name of the group `YEARMONTH`.

Let's build on this and see how we can create a DataFrame where we calculate the mean values for all those weather attributes that we were interested in. We will repeat some of the earlier steps here so you can see and better understand what is happening.

In [46]:
# Create an empty DataFrame for the aggregated values
monthly_data = pd.DataFrame()

# The columns that we want to aggregate
mean_cols = ["PRECIP", "Temp_Average_F", "TEMP_C"]

# Iterate over the groups
for key, group in grouped:

    # Calculate mean
    mean_values = group[mean_cols].mean()

    # Add the ´key´ (i.e. the date+time information) into the aggregated values
    mean_values["YEAR_MONTH"] = key
    
    # Convert the mean_values series to a DataFrame and make it have a row orientation
    row = mean_values.to_frame().transpose()

    # Concatenate the aggregated values into the monthly_data DataFrame
    monthly_data = pd.concat([monthly_data, row], ignore_index=True)

```{note}
You can find a slightly more efficient approach for this same kind of aggregation in [Chapter 3 of the *Python for Geographic Data Analysis*](https://python-gis-book.readthedocs.io/en/latest/part1/chapter-03/nb/02-data-analysis.html#aggregating-data-with-groupby) textbook.
```

Now, let us see what we have.

In [47]:
print(monthly_data)

       PRECIP Temp_Average_F     TEMP_C YEAR_MONTH
0   51.752037      55.593683  13.107602     201801
1   43.766144      54.020065  12.233369     201802
2   42.325834      47.131625   8.406458     201803
3   53.491379      48.403075   9.112819     201804
4    47.47626       49.51247    9.72915     201805
5   42.794952      45.691624   7.606458     201806
6   46.533055      53.371777   11.87321     201807
7   49.014621      53.210426   11.78357     201808
8   51.166668      50.652846  10.362692     201809
9   52.380422      46.579807   8.099893     201810
10    47.7975      46.149822   7.861012     201811
11  50.233052      52.425543  11.347524     201812
12  49.028969      42.354704   5.752613     201901
13  53.266617      53.158862  11.754924     201902
14  60.036894      56.946362   13.85909     201903
15  55.804043       46.95061   8.305895     201904
16  43.928275      50.676144  10.375635     201905
17  44.651632      49.223029    9.56835     201906
18  52.489306      49.490115   

Awesome! Now we have aggregated our data and we have a new DataFrame called `monthly_data` where we have mean values for each month in the data set.

## Detecting warm months

Now that we have aggregated our data on monthly level, all we need to do is to sort our results in order to check which years had the warmest April temperatures. A simple approach is to select all Aprils from the data, group the data and check which group(s) have the highest mean value.

We can start this by selecting all records that are from April (regardless of the year).

In [48]:
Julys = data[data["MONTH"] == "07"]

Next, we can take a subset of columns that might contain interesting information.

In [49]:
Julys = Julys[[ "Temp_Average_F", "TEMP_C", "YEAR_MONTH"]]

We can group by year and month.

In [50]:
grouped = Julys.groupby(by="YEAR_MONTH")

And then we can calculate the mean for each group.

In [51]:
monthly_mean = grouped.mean()

In [52]:
monthly_mean.head()

,Temp_Average_F,TEMP_C
YEAR_MONTH,,
201807,53.371777,11.873210
201907,49.490115,9.716731
202007,46.239410,7.910783


Finally, we can sort and check the highest temperature values. We can sort the data frame in a descending order to do this.

In [53]:
monthly_mean.sort_values(by="TEMP_C", ascending=False).head(10)

,Temp_Average_F,TEMP_C
YEAR_MONTH,,
201807,53.371777,11.873210
201907,49.490115,9.716731
202007,46.239410,7.910783


So, how did July 2018 rank at the Stockholm observation station? 

## Repeating the data analysis with a larger dataset

To wrap up today's lesson, let's repeat the data analysis steps above for all the available data we have (!). First, it would be good to confirm the path to the directory where all the input data are located.

The idea is, that we will repeat the analysis process for each input file using a (rather long) for loop! Here we have all the main analysis steps with some additional output info, all in one long code cell.

In [54]:
# Read selected columns of  data using varying amount of spaces as separator and specifying * characters as NoData values
data = pd.read_csv(
    fp,
    delim_whitespace=True,
    usecols=["YRMODAHRMN","Temperature", "Precipitation","City"],
    na_values=["*", "**", "***", "****", "*****", "******"],
)

# Rename the columns
new_names = {"Temperature": "Temp_Average_F", "Precipitation": "PRECIP", "City": "Location","YRMODAHRMN": "Date"}

data = data.rename(columns=new_names)

# Print info about the current input file:
print(f"Location: {data.at[0, 'Location']}")
print(f"NUMBER OF OBSERVATIONS: {len(data)}")

# Create column
col_name = "TEMP_C"
data[col_name] = None

# Convert tempetarues from Fahrenheits to Celsius
data["TEMP_C"] = data["Temp_Average_F"].apply(fahr_to_celsius)

# Convert TIME to string
data["Date_STR"] = data["Date"].astype(str)

# Parse year and month
data["MONTH"] = data["Date_STR"].str.slice(start=5, stop=6).astype(int)
data["YEAR"] = data["Date_STR"].str.slice(start=0, stop=4).astype(int)

# Extract observations for the months of April
Julys = data[data["MONTH"] == 7]

# Take a subset of columns
Julys = Julys[[ "Temp_Average_F", "TEMP_C", "YEAR", "MONTH"]]

# Group by year and month
grouped = Julys.groupby(by=["YEAR", "MONTH"])

# Get mean values for each group
monthly_mean = grouped.mean()

# Print info
print(monthly_mean.sort_values(by="TEMP_C", ascending=False).head(5))
print("\n")

Location: Stockholm
NUMBER OF OBSERVATIONS: 1096
            Temp_Average_F     TEMP_C
YEAR MONTH                           
2018 7           53.371777  11.873210
2019 7           49.490115   9.716731
2020 7           46.239410   7.910783




At this point we will use the `glob()` function from the module `glob` to list our input files. glob is a handy function for finding files in a directrory that match a given pattern, for example.

In [55]:
import glob

In [62]:
file_list = glob.glob(r"D:/Teaching_Development/Python_1_course_VT24/missing lecture/dataset/*txt")

```{note}
Note that we're using the \* character as a wildcard, so any file that starts with `data/0` and ends with `txt` will be added to the list of files we will iterate over. We specifically use `data/0` as the starting part of the file names to avoid having our metadata files included in the list!
```

````{note}
If you are using Jupyter Lab installed on your own computer, then the `file_list` variable you should use is defined below:

```python
file_list = glob.glob(r"data/0*txt")
```
````

In [63]:
print(f"Number of files in the list: {len(file_list)}")
print(file_list)

Number of files in the list: 5
['D:/Teaching_Development/Python_1_course_VT24/missing lecture/dataset\\Stockholm_continuous_final.txt', 'D:/Teaching_Development/Python_1_course_VT24/missing lecture/dataset\\Vasteras_continuous_final.txt', 'D:/Teaching_Development/Python_1_course_VT24/missing lecture/dataset\\Malmo_continuous_final.txt', 'D:/Teaching_Development/Python_1_course_VT24/missing lecture/dataset\\Uppsala_continuous_final.txt', 'D:/Teaching_Development/Python_1_course_VT24/missing lecture/dataset\\Gothenburg_continuous_final.txt']


Now, you should have all the relevant file names in a list, and we can loop over the list using a for loop.

In [64]:
for fp in file_list:
    print(fp)

D:/Teaching_Development/Python_1_course_VT24/missing lecture/dataset\Stockholm_continuous_final.txt
D:/Teaching_Development/Python_1_course_VT24/missing lecture/dataset\Vasteras_continuous_final.txt
D:/Teaching_Development/Python_1_course_VT24/missing lecture/dataset\Malmo_continuous_final.txt
D:/Teaching_Development/Python_1_course_VT24/missing lecture/dataset\Uppsala_continuous_final.txt
D:/Teaching_Development/Python_1_course_VT24/missing lecture/dataset\Gothenburg_continuous_final.txt


In [65]:
# Repeat the analysis steps for each input file:
for fp in file_list:

    # Read selected columns of  data using varying amount of spaces as separator and specifying * characters as NoData values
    data = pd.read_csv(
        fp,
        delim_whitespace=True,
        usecols=["YRMODAHRMN","Temperature", "Precipitation","City"],
        na_values=["*", "**", "***", "****", "*****", "******"],
    )

    # Rename the columns
    new_names = {"Temperature": "Temp_Average_F", "Precipitation": "PRECIP", "City": "Location","YRMODAHRMN": "Date"}

    data = data.rename(columns=new_names)

    # Print info about the current input file:
    print(f"Location: {data.at[0, 'Location']}")
    print(f"NUMBER OF OBSERVATIONS: {len(data)}")

    # Create column
    col_name = "TEMP_C"
    data[col_name] = None

    # Convert tempetarues from Fahrenheits to Celsius
    data["TEMP_C"] = data["Temp_Average_F"].apply(fahr_to_celsius)

    # Convert TIME to string
    data["Date_STR"] = data["Date"].astype(str)

    # Parse year and month
    data["MONTH"] = data["Date_STR"].str.slice(start=5, stop=6).astype(int)
    data["YEAR"] = data["Date_STR"].str.slice(start=0, stop=4).astype(int)

    # Extract observations for the months of April
    Julys = data[data["MONTH"] == 7]

    # Take a subset of columns
    Julys = Julys[[ "Temp_Average_F", "TEMP_C", "YEAR", "MONTH"]]

    # Group by year and month
    grouped = Julys.groupby(by=["YEAR", "MONTH"])

    # Get mean values for each group
    monthly_mean = grouped.mean()

    # Print info
    print(monthly_mean.sort_values(by="TEMP_C", ascending=False).head(5))
    print("\n")

Location: Stockholm
NUMBER OF OBSERVATIONS: 1096
            Temp_Average_F     TEMP_C
YEAR MONTH                           
2018 7           53.371777  11.873210
2019 7           49.490115   9.716731
2020 7           46.239410   7.910783


Location: Vasteras
NUMBER OF OBSERVATIONS: 1096
            Temp_Average_F     TEMP_C
YEAR MONTH                           
2018 7           52.178709  11.210394
2019 7           48.292504   9.051391
2020 7           46.981839   8.323244


Location: Malmo
NUMBER OF OBSERVATIONS: 1096
            Temp_Average_F     TEMP_C
YEAR MONTH                           
2018 7           52.746521  11.525845
2019 7           48.382259   9.101255
2020 7           46.622180   8.123433


Location: Uppsala
NUMBER OF OBSERVATIONS: 1096
            Temp_Average_F     TEMP_C
YEAR MONTH                           
2018 7           52.493039  11.385022
2019 7           50.085999  10.047777
2020 7           45.959570   7.755317


Location: Gothenburg
NUMBER OF OBSERVATIONS